In [2]:
# ---------------------------------------------------------------
# 🧠 WUMPUS WORLD ENTAILMENT CHECKER USING PROPOSITIONAL LOGIC
# ---------------------------------------------------------------
# Method: Resolution
# Goal: Check if Knowledge Base (KB) ⊨ Query (Q)
# ---------------------------------------------------------------

# --- Helper functions ---

def negate(literal):
    """
    Negate a literal.
    Example: 'P_1_2' → '¬P_1_2' ,  '¬P_1_2' → 'P_1_2'
    """
    return literal[1:] if literal.startswith('¬') else '¬' + literal


def resolve(ci, cj):
    """
    Apply resolution rule between two clauses ci and cj.
    Returns a set of all possible resolvents.
    """
    resolvents = set()
    for li in ci:
        if negate(li) in cj:
            # Remove complementary literals and combine remaining
            new_clause = (ci.union(cj)) - {li, negate(li)}
            resolvents.add(frozenset(new_clause))
    return resolvents


def entails(KB, query):
    """
    Check if KB entails the query using the Resolution Algorithm.
    Returns True if KB ⊨ query, otherwise False.
    """
    # Copy KB and add the negated query
    clauses = KB.copy()
    clauses.append({negate(query)})

    new = set()
    while True:
        n = len(clauses)
        # Generate all unique pairs of clauses
        pairs = [(clauses[i], clauses[j]) for i in range(n) for j in range(i + 1, n)]

        for (ci, cj) in pairs:
            resolvents = resolve(ci, cj)

            # If the empty clause is found → contradiction → entailment holds
            if frozenset() in resolvents:
                return True

            new |= resolvents  # accumulate new resolvents

        # If no new information, stop
        new_clauses = [set(c) for c in new if set(c) not in clauses]
        if not new_clauses:
            return False  # no contradiction → not entailed

        # Extend KB with new clauses
        clauses.extend(new_clauses)


# ---------------------------------------------------------------
# 🧩 Knowledge Base (CNF Clauses)
# ---------------------------------------------------------------
# Represent each clause as a set of literals (strings)
# Example world: 2x2 grid
# Observations:
#   B_1_1 = True  (breeze at 1,1)
#   B_2_1 = False (no breeze at 2,1)
# Rules:
#   B_1_1 ↔ (P_1_2 ∨ P_2_1)
#   ¬B_2_1 → ¬P_2_1
# ---------------------------------------------------------------

KB = [
    {'B_1_1'},                          # (1) Breeze at (1,1)
    {'¬B_2_1'},                         # (2) No breeze at (2,1)
    {'¬B_1_1', 'P_1_2', 'P_2_1'},       # (3) Breeze rule (B_1_1 → P_1_2 ∨ P_2_1)
    {'B_2_1', '¬P_2_1'}                 # (4) No breeze → no pit at (2,1)
]

# ---------------------------------------------------------------
# 🔎 Test Queries
# ---------------------------------------------------------------

queries = ['P_1_2', 'W_2_2']  # Query 1: Pit at (1,2)?  Query 2: Wumpus at (2,2)?

print("\n🧠 WUMPUS WORLD ENTAILMENT CHECK (Resolution Method)")
print("-----------------------------------------------------")

for q in queries:
    result = entails(KB, q)
    print(f"Query: {q:6s} → {'✅ ENTAILED (True)' if result else '❌ NOT ENTAILED (False)'}")

print("\nExplanation:")
print("  ✔ P_1_2 is entailed because breeze at (1,1) and no pit at (2,1) ⇒ pit at (1,2)")
print("  ❌ W_2_2 is not entailed because there is no information about stench/Wumpus\n")



🧠 WUMPUS WORLD ENTAILMENT CHECK (Resolution Method)
-----------------------------------------------------
Query: P_1_2  → ✅ ENTAILED (True)
Query: W_2_2  → ❌ NOT ENTAILED (False)

Explanation:
  ✔ P_1_2 is entailed because breeze at (1,1) and no pit at (2,1) ⇒ pit at (1,2)
  ❌ W_2_2 is not entailed because there is no information about stench/Wumpus



In [3]:
# ---------------------------------------------------------------
# 🧠 WUMPUS WORLD ENTAILMENT CHECKER (DETAILED VERSION)
# ---------------------------------------------------------------
# Uses: Resolution Method in Propositional Logic
# Goal: Check whether Knowledge Base (KB) entails a given Query (Q)
# ---------------------------------------------------------------

def negate(literal):
    """Negate a literal, e.g., P_1_2 → ¬P_1_2, ¬P_1_2 → P_1_2."""
    return literal[1:] if literal.startswith('¬') else '¬' + literal


def resolve(ci, cj):
    """
    Apply resolution between two clauses ci and cj.
    Returns a set of all possible resolvents.
    """
    resolvents = set()
    for li in ci:
        if negate(li) in cj:
            new_clause = (ci.union(cj)) - {li, negate(li)}
            resolvents.add(frozenset(new_clause))
    return resolvents


def entails_detailed(KB, query):
    """
    Perform propositional resolution and print detailed steps.
    """
    print(f"\n{'='*60}")
    print(f"🔍 Checking entailment for query: {query}")
    print(f"{'='*60}\n")

    # Show initial KB
    print("📘 Initial Knowledge Base (CNF Clauses):")
    for i, c in enumerate(KB, 1):
        print(f"  ({i}) {c}")
    print()

    # Add negated query to KB
    neg_q = {negate(query)}
    clauses = KB.copy()
    clauses.append(neg_q)
    print(f"➕ Added negated query: {neg_q}\n")

    new = set()
    step = 1

    while True:
        n = len(clauses)
        pairs = [(clauses[i], clauses[j]) for i in range(n) for j in range(i + 1, n)]

        for (ci, cj) in pairs:
            resolvents = resolve(ci, cj)

            for r in resolvents:
                print(f"Step {step}: Resolve {ci} and {cj}  →  {set(r)}")
                step += 1

                # If empty clause (contradiction) is found
                if not r:
                    print("\n🚨 CONTRADICTION FOUND (empty clause derived)!")
                    print(f"✅ Therefore, KB entails {query}\n")
                    return True

            new |= resolvents

        # Stop if no new information
        new_clauses = [set(c) for c in new if set(c) not in clauses]
        if not new_clauses:
            print("\n❌ No contradiction found.")
            print(f"KB does NOT entail {query}\n")
            return False

        clauses.extend(new_clauses)


# ---------------------------------------------------------------
# 🧩 Knowledge Base (CNF Clauses)
# ---------------------------------------------------------------
KB = [
    {'B_1_1'},                          # Breeze at (1,1)
    {'¬B_2_1'},                         # No breeze at (2,1)
    {'¬B_1_1', 'P_1_2', 'P_2_1'},       # Breeze rule: B_1_1 → P_1_2 ∨ P_2_1
    {'B_2_1', '¬P_2_1'}                 # No breeze → no pit at (2,1)
]

# ---------------------------------------------------------------
# 🔎 Test Queries
# ---------------------------------------------------------------
queries = ['P_1_2', 'W_2_2']

print("\n🧠 WUMPUS WORLD ENTAILMENT CHECK (DETAILED MODE)")
print("-------------------------------------------------\n")

for q in queries:
    entails_detailed(KB, q)



🧠 WUMPUS WORLD ENTAILMENT CHECK (DETAILED MODE)
-------------------------------------------------


🔍 Checking entailment for query: P_1_2

📘 Initial Knowledge Base (CNF Clauses):
  (1) {'B_1_1'}
  (2) {'¬B_2_1'}
  (3) {'P_2_1', '¬B_1_1', 'P_1_2'}
  (4) {'B_2_1', '¬P_2_1'}

➕ Added negated query: {'¬P_1_2'}

Step 1: Resolve {'B_1_1'} and {'P_2_1', '¬B_1_1', 'P_1_2'}  →  {'P_2_1', 'P_1_2'}
Step 2: Resolve {'¬B_2_1'} and {'B_2_1', '¬P_2_1'}  →  {'¬P_2_1'}
Step 3: Resolve {'P_2_1', '¬B_1_1', 'P_1_2'} and {'B_2_1', '¬P_2_1'}  →  {'¬B_1_1', 'B_2_1', 'P_1_2'}
Step 4: Resolve {'P_2_1', '¬B_1_1', 'P_1_2'} and {'¬P_1_2'}  →  {'P_2_1', '¬B_1_1'}
Step 5: Resolve {'B_1_1'} and {'P_2_1', '¬B_1_1', 'P_1_2'}  →  {'P_2_1', 'P_1_2'}
Step 6: Resolve {'B_1_1'} and {'P_2_1', '¬B_1_1'}  →  {'P_2_1'}
Step 7: Resolve {'B_1_1'} and {'¬B_1_1', 'B_2_1', 'P_1_2'}  →  {'B_2_1', 'P_1_2'}
Step 8: Resolve {'¬B_2_1'} and {'B_2_1', '¬P_2_1'}  →  {'¬P_2_1'}
Step 9: Resolve {'¬B_2_1'} and {'¬B_1_1', 'B_2_1', 'P_1_2'}  

In [4]:
# ---------------------------------------------------------------
# 🧠 WUMPUS WORLD INFERENCE SYSTEM
# ---------------------------------------------------------------
# Implements the algorithm described for logical inference
# using percepts (Breeze, Stench, Glitter)
# ---------------------------------------------------------------

# Helper: check valid grid cell
def neighbors(x, y, N):
    """Return valid neighbor coordinates within NxN grid."""
    possible = [(x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1)]
    return [(i, j) for (i, j) in possible if 1 <= i <= N and 1 <= j <= N]


def print_grid(Grid, N):
    """Display current KB snapshot in grid form."""
    print("\n🧩 Current Knowledge Base Snapshot:")
    for y in range(N, 0, -1):  # print top to bottom
        row = []
        for x in range(1, N + 1):
            cell = Grid[x][y]
            if not cell["V"]:
                row.append("🟦")  # Unvisited cell
            else:
                pit = "P?" if cell["P"] == "Possible" else ("¬P" if cell["P"] is False else ("P" if cell["P"] else ""))
                wumpus = "W?" if cell["W"] == "Possible" else ("¬W" if cell["W"] is False else ("W" if cell["W"] else ""))
                content = f"{pit}{wumpus}" if (pit or wumpus) else "🟩"
                row.append(content.center(6))
        print(" | ".join(row))
    print()


def Wumpus_Inference(N, Percepts):
    # 1. Initialize world grid
    Grid = {x: {y: {"S": None, "B": None, "G": None, "P": None, "W": None, "V": False}
                for y in range(1, N + 1)} for x in range(1, N + 1)}

    # 2. Initialize Knowledge Base (KB)
    KB = []

    # 3. Define rules implicitly applied in the loop (see algorithm)
    print("🔰 Starting Wumpus World Inference System\n")

    # 5. Agent start position
    Agent_Start = (1, 1)
    Path = [(1, 1), (2, 1), (1, 2), (2, 2), (3, 2)]

    for step, (x, y) in enumerate(Path, start=1):
        print(f"\n🚶 Step {step}: Agent moves to cell ({x},{y})")
        Grid[x][y]["V"] = True

        # (c) Get percepts
        percept = Percepts.get((x, y), {"S": None, "B": None, "G": None})
        Grid[x][y]["S"] = percept["S"]
        Grid[x][y]["B"] = percept["B"]
        Grid[x][y]["G"] = percept["G"]

        # (d) Update KB with percepts
        KB.append(f"S({x},{y})={percept['S']}")
        KB.append(f"B({x},{y})={percept['B']}")

        # (e) Rule 5: current cell is safe
        Grid[x][y]["P"] = False
        Grid[x][y]["W"] = False

        # (f) Apply inference rules
        adj = neighbors(x, y, N)

        # Breeze logic
        if percept["B"] is True:
            print(f"💨 Breeze detected at ({x},{y}) → Possible Pit in neighbors.")
            for (i, j) in adj:
                if Grid[i][j]["P"] is None:
                    Grid[i][j]["P"] = "Possible"
        elif percept["B"] is False:
            print(f"✅ No Breeze at ({x},{y}) → No Pit in any adjacent cells.")
            for (i, j) in adj:
                Grid[i][j]["P"] = False

        # Stench logic
        if percept["S"] is True:
            print(f"💀 Stench detected at ({x},{y}) → Possible Wumpus in neighbors.")
            for (i, j) in adj:
                if Grid[i][j]["W"] is None:
                    Grid[i][j]["W"] = "Possible"
        elif percept["S"] is False:
            print(f"✅ No Stench at ({x},{y}) → No Wumpus in any adjacent cells.")
            for (i, j) in adj:
                Grid[i][j]["W"] = False

        # (g) Display current KB
        print_grid(Grid, N)

    # 8. Evaluate entailment for given queries
    pit12 = Grid[1][2]["P"]
    wumpus22 = Grid[2][2]["W"]

    # 9. Output final results
    print("\n🏁 Final Entailment Results:")
    if pit12 is True:
        print("Query1: Pit(1,2) ⇒ ✅ KB entails Pit at (1,2)")
    elif pit12 is False:
        print("Query1: Pit(1,2) ⇒ ❌ KB entails No Pit at (1,2)")
    else:
        print("Query1: Pit(1,2) ⇒ ❓ Uncertain about Pit at (1,2)")

    if wumpus22 is True:
        print("Query2: Wumpus(2,2) ⇒ ✅ KB entails Wumpus at (2,2)")
    elif wumpus22 is False:
        print("Query2: Wumpus(2,2) ⇒ ❌ KB entails No Wumpus at (2,2)")
    else:
        print("Query2: Wumpus(2,2) ⇒ ❓ Uncertain about Wumpus at (2,2)")

    print("\n📘 Final KB State:")
    for k in KB:
        print("  ", k)


# ---------------------------------------------------------------
# 🧩 SAMPLE PERCEPTS for a 3x3 Grid
# (B = Breeze, S = Stench, G = Glitter)
# ---------------------------------------------------------------
Percepts = {
    (1, 1): {"B": True,  "S": False, "G": False},
    (2, 1): {"B": False, "S": False, "G": False},
    (1, 2): {"B": None,  "S": None,  "G": None},
    (2, 2): {"B": None,  "S": None,  "G": None},
    (3, 2): {"B": None,  "S": None,  "G": None},
}

# Run the algorithm
Wumpus_Inference(3, Percepts)


🔰 Starting Wumpus World Inference System


🚶 Step 1: Agent moves to cell (1,1)
💨 Breeze detected at (1,1) → Possible Pit in neighbors.
✅ No Stench at (1,1) → No Wumpus in any adjacent cells.

🧩 Current Knowledge Base Snapshot:
🟦 | 🟦 | 🟦
🟦 | 🟦 | 🟦
 ¬P¬W  | 🟦 | 🟦


🚶 Step 2: Agent moves to cell (2,1)
✅ No Breeze at (2,1) → No Pit in any adjacent cells.
✅ No Stench at (2,1) → No Wumpus in any adjacent cells.

🧩 Current Knowledge Base Snapshot:
🟦 | 🟦 | 🟦
🟦 | 🟦 | 🟦
 ¬P¬W  |  ¬P¬W  | 🟦


🚶 Step 3: Agent moves to cell (1,2)

🧩 Current Knowledge Base Snapshot:
🟦 | 🟦 | 🟦
 ¬P¬W  | 🟦 | 🟦
 ¬P¬W  |  ¬P¬W  | 🟦


🚶 Step 4: Agent moves to cell (2,2)

🧩 Current Knowledge Base Snapshot:
🟦 | 🟦 | 🟦
 ¬P¬W  |  ¬P¬W  | 🟦
 ¬P¬W  |  ¬P¬W  | 🟦


🚶 Step 5: Agent moves to cell (3,2)

🧩 Current Knowledge Base Snapshot:
🟦 | 🟦 | 🟦
 ¬P¬W  |  ¬P¬W  |  ¬P¬W 
 ¬P¬W  |  ¬P¬W  | 🟦


🏁 Final Entailment Results:
Query1: Pit(1,2) ⇒ ❌ KB entails No Pit at (1,2)
Query2: Wumpus(2,2) ⇒ ❌ KB entails No Wumpus at (2,2)

📘 Final KB